# Классификация изображений природы по времени года

**Проектная работа Ситникова Андрея на курсе «Machine Learning. Professional»**

## Часть 3. Кластеризация

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
from IPython.display import HTML, display

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

np.random.seed(100)

Функция приведения к безопасному цвету из прошлой части.

In [2]:
def get_safe_color(color):
    return int(round((color / 255) * 5) * 51)

get_safe_color_vec = np.vectorize(get_safe_color)

Создаём список столбцов для датафрейма, условно обозначив каждое значение R, G и B от 0 до 5.

In [3]:
columns=['season']
for r in range(0, 256, 51):
    for g in range(0, 256, 51):
        for b in range(0, 256, 51):
            columns.append(f"c{r//51}{g//51}{b//51}")
columns[:10]

['season',
 'c000',
 'c001',
 'c002',
 'c003',
 'c004',
 'c005',
 'c010',
 'c011',
 'c012']

In [4]:
df_ml = pd.DataFrame(columns=columns)
df_ml

,season,c000,c001,c002,c003,c004,c005,c010,c011,c012,...,c542,c543,c544,c545,c550,c551,c552,c553,c554,c555


Создаём болванку для строки датафрейма с нулями у каждого значения цвета.

In [5]:
null_row = {}
for col in columns:
    null_row[col] = 0

Пробегаем в цикле по всем папкам и для каждой картинки делаем кластеризацию.

In [6]:
seasons = ['summer', 'autumn', 'winter', 'spring']
for season in seasons:
    files = os.listdir(f'img/{season}')
    for file in tqdm(files):
        try:
            # Загружаем картинку
            with Image.open(f'img/{season}/{file}') as im:
                px = im.load()            
            
            # Создаём датафрейм из всех её пикселей
            arr = []
            for i in range(im.size[0]):
                for j in range(im.size[1]):
                    r, g, b = px[i,j]
                    arr.append([r, g, b])
            df = pd.DataFrame(arr, columns=['r', 'g', 'b'])
            
            # Делаем кластеризацию с 10 кластерами
            kmeans = KMeans(n_clusters=10, n_jobs=-1).fit(df)
            
            # Сохраняем цвета центров кластеров, приводим к безопасному виду, удаляем дубликаты
            centers = kmeans.cluster_centers_
            centers = get_safe_color_vec(centers)
            centers = np.unique(centers, axis=0)
            
            # Копируем нулевую строку, проставляем единички у элементов словаря, совпадающих с центрами
            row = null_row.copy()
            row['season'] = season
            for center in centers:
                R, G, B = center
                row[f"c{R//51}{G//51}{B//51}"] = 1
            
            # Заносим получившуюся строку в датафрейм
            df_ml = df_ml.append(row, ignore_index=True)
        except:
            print(f'Ошибка обработки файла img/{season}/{file}')
df_ml

100%|██████████| 596/596 [1:38:26<00:00,  9.91s/it]


,season,c000,c001,c002,c003,c004,c005,c010,c011,c012,...,c542,c543,c544,c545,c550,c551,c552,c553,c554,c555
0,summer,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,summer,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,summer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,summer,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,summer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,spring,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2374,spring,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2375,spring,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2376,spring,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


Удаляем из датафрейма одинаковые строки, если таковые получились.

In [7]:
df_ml = df_ml.drop_duplicates()
df_ml

,season,c000,c001,c002,c003,c004,c005,c010,c011,c012,...,c542,c543,c544,c545,c550,c551,c552,c553,c554,c555
0,summer,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,summer,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,summer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,summer,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,summer,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2373,spring,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2374,spring,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2375,spring,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2376,spring,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


Сохраняем датафрейм.

In [8]:
df_ml.to_csv('df_ml.csv', index=False)